# A Portuguese Merchant Network in the Initial Times of the Manila Galleon System (1565)

In [1]:
from collections import Counter
import re

import networkx as nx
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 100)

## Loading data

In [3]:
col_names = [
    'NAME',
    'COUNTRY_ORIGIN',
    'LOCATION',
    'PROFESSION_EDUCATION',
    'RELATION',
    'CONTACTS',
    'INQUISITION_TRIAL',
    'SENTENCES',
]
df = pd.read_excel('data/src/NetworkParticipants.xlsx', names=col_names)
df.fillna('', inplace=True)

In [4]:
df.shape

(84, 8)

In [5]:
df.head()

,NAME,COUNTRY_ORIGIN,LOCATION,PROFESSION_EDUCATION,RELATION,CONTACTS,INQUISITION_TRIAL,SENTENCES
0,"Almeida, Jorge de",Portugal,"Ferrara, Pachuca, Madrid","Business partner of Díaz de Cáceres, Mines of ...","Invested 1,600 in the ship Nuestra Señora de l...",,,
1,"Anbrozio, Mestre Xrováo",Portugal,Canton,Merchant,Business Partner with Diege Hernández Victoria...,,,
2,"Andrada (Carvajal), Leonor de",Spain,Mexico,Housewife,Married to Joge de Almeida,"António Díaz de Cáceres, Luis de Carvajal, Lui...","AGN, Inquisición, vol.12, exp 2.",
3,"Baeça, Pedro",Portugal,"Goa, Melaka, Macao, Nagasaki, Manila",Official /Ship Pilot / Shipowner / Merchant,Business Partner with BVL,Bartolomé Báez Landero,,
4,"Báez Landero, Bartolomé. (Bartolomeu Vaz Lande...",Portugal,"Macao, Manila",Merchant / Shipowner/,Business Partner,"Sebastiāo Jorge, Antonio Vaz, Henrique Borges,...",,


## Preprocessing data

In [6]:
df2 = pd.DataFrame()

In [7]:
def name_and_notes(x):
    x = x.strip('*')
    notes = re.findall('\(.+\)', x)
    if not notes:
        notes = ''
        name = x
    else:
        notes = notes[0]
        name = x.replace(notes, '')
    notes = notes.strip('()')
    if ',' in name:
        last = name.split(',')[0].strip(' .')
        first = name.split(',')[1].strip(' .')
    else:
        first, last = name.split()
    fullname = first + ' ' + last
    return pd.Series((fullname, notes, first, last))

df2[['FULLNAME', 'NAME_NOTES', 'FIRST_NAME', 'LAST_NAME']] = df.NAME.apply(name_and_notes)

In [8]:
def country_and_notes(x):
    x = x.strip()
    if x == 'Braganza':
        country = 'Portugal'
        notes = 'Bragança'
        place = 'Braganza'
    elif x == 'Canaries':
        country = 'Spain'
        notes = ''
        place = 'Canaries'
    elif x == 'Freixo de Espadacinta, Portugal':
        country = 'Portugal'
        notes = ''
        place = 'Freixo de Espadacinta'
    elif x == 'Jerez de la Frontera':
        country = 'Spain'
        notes = ''
        place = 'Jerez de la Frontera'
    elif x == 'Jerez de la Frontera, Spain (Tunez)':
        country = 'Spain'
        notes = 'Tunez'
        place = 'Jerez de la Frontera'
    elif x == 'Lisbon':
        country = 'Portugal'
        notes = ''
        place = 'Lisbon'
    elif x == 'Portugal ?':
        country = 'Portugal'
        notes = '?'
        place = ''
    elif x == 'Province of La Guarda':
        country = 'Portugal'
        notes = ''
        place = 'Province of La Guarda'
    elif x == 'San Vicente Davera, Province of La Guarda':
        country = 'Portugal'
        notes = 'San Vicente Davera'
        place = 'Province of La Guarda'
    elif x == 'Santa Comba Dāo, Portugal':
        country = 'Portugal'
        notes = ''
        place = 'Santa Comba Dāo'
    elif x == 'Spain (?)':
        country = 'Spain'
        notes = '?'
        place = ''
    elif x == 'Spain ?':
        country = 'Spain'
        notes = '?'
        place = ''
    elif x == 'Spain?':
        country = 'Spain'
        notes = '?'
        place = ''
    elif x == 'Unknown':
        country = ''
        notes = ''
        place = ''
    elif x == 'Viseu, Portugal':
        country = 'Portugal'
        notes = ''
        place = 'Viseu'
    else:
        country = x
        notes = ''
        place = ''
        
    return pd.Series((country, notes, place))

df2[['COUNTRY_ORIGIN', 'COUNTRY_NOTES', 'PLACE_ORIGIN']] = df.COUNTRY_ORIGIN.apply(country_and_notes)

In [9]:
def location_and_notes(x):
    x = x.strip()
    if x == 'Cartagena , Mexico (1594)':
        location = 'Cartagena, Mexico'
        notes = '1594'
    elif x == 'Goa, Macao, Manila (1595-1597)':
        location = 'Goa, Macao, Manila'
        notes = '1595-1597'
    elif x == 'Manila / Japan':
        location = 'Manila, Japan'
        notes = ''
    elif x == 'Manila / Mexico':
        location = 'Manila, Mexico'
        notes = ''
    elif x == 'Manila Mexico':
        location = 'Manila, Mexico'
        notes = ''
    elif x == 'Melaka, 1595':
        location = 'Melaka'
        notes = '1595'
    elif x == 'Tlahuililpa, a mine near Pachuca':
        location = 'Tlahuililpa'
        notes = 'A mine near Pachuca'
    else:
        location = x
        notes = ''
    return pd.Series((location, notes))

df2[['LOCATION', 'LOC_NOTES']] = df.LOCATION.apply(location_and_notes)

In [10]:
def prof_edu_rel_notes(x):
    y = x.PROFESSION_EDUCATION.strip()
    z = x.RELATION.strip()
    w = x.FULLNAME
    if y == 'Ship Pilot' and z == 'Nephew of Bartolomeu Vaz Landeiro  / Legal Proxy of BVL':
        prof_edu = 'Shipman'
        relation = 'Bartolomé Báez Landero++uncle of--' + w + ', legal proxy of--Bartolomé Báez Landero'
        notes = ''
    elif y == 'Administrator/ Superintendent /' and z == 'Business Partner of Feliciano Valencia, Antonio Díaz de Cáceres':
        prof_edu = 'Official'
        relation = 'partner of--Feliciano Valencia, partner of--Antonio Díaz de Cáceres'
        notes = 'Superintendent'
    elif y == 'Business Partner with Antonio Díaz de Cáceres, 1589' and z == 'Invested 4,000 pesos in the galleon Nuestra Señora de la Concepción bound to Manila, 1589.':
        prof_edu = 'Merchant'
        relation = 'partner of--Antonio Díaz de Cáceres, invested in--Nuestra Señora de la Concepción'#--4000'
        notes = 'Manila, 1589.'
    elif y == 'Business partner of Díaz de Cáceres, Mines of Taxco' and z == 'Invested 1,600 in the ship Nuestra Señora de la Concepción, 1589':
        prof_edu = 'Merchant'
        relation = 'partner of--Antonio Díaz de Cáceres, invested in--Nuestra Señora de la Concepción'#--1600'
        notes = 'Mines of Taxco. Manila, 1589'
    elif y == 'Captain' and z == 'DHV paid 35 pesos to Joan de Alzesa':
        prof_edu = 'Shipman'
        relation = 'Diego Hernández Victoria++paid to--' + w# + '--35'
        notes = ''
    elif y == 'Captain / Shipowner / Landowner / Merchant' and z == 'Partner of Diego Hernandez Victoria':
        prof_edu = 'Shipman, Landowner, Merchant'
        relation = 'partner of--Diego Hernández Victoria'
        notes = ''
    elif y == 'Captain Major Macao -Kochinotsu (1582) / Merchant' and z == 'Legal Proxy BVL, Business Partner Diego Hernández Victoria,':
        prof_edu = 'Captain-Major, Merchant'
        relation = 'captain-major of--Macao, legal proxy of--Bartolomé Báez Landero, partner of--Diego Hernández Victoria'
        notes = 'Kochinotsu (1582)'
    elif y == 'Captain Manila Galleon' and z == 'DHV paid a security deposit for property of Luis de Vivanco':
        prof_edu = 'Shipman'
        relation = 'captain of--Manila Galleon, Diego Hernández Victoria++paid to--' + w# + '--0'
        notes = 'DHV paid a security deposit for property of Luis de Vivanco'
    elif y == 'Captain Manila Galleon' and z == 'Executor of the will of Diego Hernández Victoria':
        prof_edu = 'Shipman'
        relation = 'captain of--Manila Galleon, executor of the will of--Diego Hernández Victoria'
        notes = ''
    elif y == 'Governor of Nuevo Reino de León' and z == '':
        prof_edu = 'Governor'
        relation = 'governor of--Nuevo Reino de León'
        notes = ''
    elif y == 'Housewife' and z == 'Married to Joge de Almeida':
        prof_edu = 'Housewife'
        relation = 'spouse of--Jorge de Almeida'
        notes = ''
    elif y == 'Housewife' and z == 'Wife of Manuel de Lucena':
        prof_edu = 'Housewife'
        relation = 'spouse of--Manuel de Lucena'
        notes = ''
    elif y == 'In charge of the royal warehouse of Manila' and z == 'Salcedo gave 100 bushels (fanegas) of rice to DHV':
        prof_edu = 'Official'
        relation = 'officer in--Manila, paid to--Diego Hernández Victoria'#--100--bushel--rice'
        notes = 'In charge of the royal warehouse of Manila'
    elif y == 'Merc hant' and z == 'Executor of the will of Diego Hernández Victoria':
        prof_edu = 'Merchant'
        relation = 'executor of the will of--Diego Hernández Victoria'
        notes = ''
    elif y == 'Merchant' and z == '':
        prof_edu = 'Merchant'
        relation = ''
        notes = ''
    elif y == 'Merchant' and z == 'Alonso Cano owns 50 pesos to DHV':
        prof_edu = 'Merchant'
        relation = 'owes to--Diego Hernández Victoria'#--50'
        notes = ''
    elif y == 'Merchant' and z == 'Alonso Torres received power of attorney from DHV':
        prof_edu = 'Merchant'
        relation = 'Diego Hernández Victoria++granted power of attorney to--' + w
        notes = ''
    elif y == 'Merchant' and z == 'Brother-in-law of Jorge de Almeida and Antonio Díaz de Cáceres':
        prof_edu = 'Merchant'
        relation = 'brother-in-law of--Jorge de Almeida, brother-in-law of--Antonio Díaz de Cáceres'
        notes = ' = '''
    elif y == 'Merchant' and z == 'Business Partner with Diege Hernández Victoria (letter of Joāo de Oliveira 1584)':
        prof_edu = 'Merchant'
        relation = 'partner of--Diego Hernández Victoria'
        notes = 'Letter of Joāo Oliveira, 1584'
    elif y == 'Merchant' and z == 'Business Partner with Joāo Oliveira (1584)':
        prof_edu = 'Merchant'
        relation = 'partner of--Joāo Oliveira'
        notes = '1584'
    elif y == 'Merchant' and z == 'Business partner with Diego Fernandez Victoria':
        prof_edu = 'Merchant'
        relation = 'partner of--Diego Hernández Victoria'
        notes = ''
    elif y == 'Merchant' and z == 'Business partner with Diego Fernandez Victoria [Letter from Canton, 1584]':
        prof_edu = 'Merchant'
        relation = 'partner of--Diego Hernández Victoria'
        notes = 'Letter from Canton, 1584'
    elif y == 'Merchant' and z == 'Cousin of Diego Fernandez Victoria':
        prof_edu = 'Merchant'
        relation = 'cousin of--Diego Hernández Victoria'
        notes = ''
    elif y == 'Merchant' and z == 'Diego Hernández Victoria lend him 4,000 pesos':
        prof_edu = 'Merchant'
        relation = 'owes to--Diego Hernández Victoria'#--4000'
        notes = ''
    elif y == 'Merchant' and z == 'Diego Hernández Victoria paid 790 pesos to pay the Chinese merchants for the trip to Japan':
        prof_edu = 'Merchant'
        relation = 'Diego Hernández Victoria++paid to--' + w# + '--790'
        notes = 'Payment to the Chinese merchants for the trip to Japan'
    elif y == 'Merchant' and z == 'Diego Hernández Victoria recieved 1,300 pesos':
        prof_edu = 'Merchant'
        relation = 'paid to--Diego Hernández Victoria'#--1300'
        notes = ''
    elif y == 'Merchant' and z == 'Diego Hernández Victoria sent merchandise with him to Francisco de Paz':
        prof_edu = 'Merchant'
        relation = 'sent merchandise to--Francisco de Paz, Diego Hernández Victoria++sent merchandise to--Francisco de Paz'
        notes = 'With Diego Hernández Victoria'
    elif y == 'Merchant' and z == 'Executor of the will of Diego Hernández Victoria':
        prof_edu = 'Merchant'
        relation = 'executor of the will of--Diego Hernández Victoria'
        notes = ''
    elif y == 'Merchant' and z == 'Legal Proxy BVL':
        prof_edu = 'Merchant'
        relation = 'legal proxy of--Bartolomé Báez Landero'
        notes = ''
    elif y == 'Merchant' and z == 'Legal Proxy of BVL':
        prof_edu = 'Merchant'
        relation = 'legal proxy of--Bartolomé Báez Landero'
        notes = ''
    elif y == 'Merchant' and z == 'Loss of merchandises with the ship St. Felipe in Nagasaki, 1597. Contaduria 1200, fl 717.':
        prof_edu = 'Merchant'
        relation = ''
        notes = 'Loss of merchandises with the ship St. Felipe in Nagasaki, 1597. Contaduria 1200, fl 717.'
    elif y == 'Merchant' and z == 'Partner of Diego Hernandez Victoria':
        prof_edu = 'Merchant'
        relation = 'partner of--Diego Hernández Victoria'
        notes = ''
    elif y == 'Merchant' and z == 'Partner of Fernando de Castro':
        prof_edu = 'Merchant'
        relation = 'partner of--Fernando de Castro'
        notes = ''
    elif y == 'Merchant' and z == 'Partner of Martín de Saravia, Fernando de Castro':
        prof_edu = 'Merchant'
        relation = 'partner of--Martín de Saravia, partner of--Fernando de Castro'
        notes = ''
    elif y == 'Merchant' and z == 'Partner with Diego Hernandez Victoria':
        prof_edu = 'Merchant'
        relation = 'partner of--Diego Hernández Victoria'
        notes = ''
    elif y == 'Merchant' and z == 'Partner with Diego Hernández Victoria':
        prof_edu = 'Merchant'
        relation = 'partner of--Diego Hernández Victoria'
        notes = ''
    elif y == 'Merchant' and z == 'Partner with Joāo Oliveira (1584)':
        prof_edu = 'Merchant'
        relation = 'partner of--Joāo Oliveira'
        notes = '1584'
    elif y == 'Merchant' and z == 'Servant of Jorge de Almeida':
        prof_edu = 'Merchant'
        relation = 'servant of--Jorge de Almeida'
        notes = ''
    elif y == 'Merchant' and z == 'Travelled to  Manila with Antonio Díaz de Cáceres, 1589':
        prof_edu = 'Merchant'
        relation = 'travelled to--Manila, travelled with--Antonio Díaz de Cáceres'
        notes = '1589'
    elif y == 'Merchant' and z == 'Uncle of Tomás de Fonseca and Héctor de Fonseca':
        prof_edu = 'Merchant'
        relation = 'uncle of--Tomás de Fonseca Castellano, uncle of--Héctor de Fonseca'
        notes = ''
    elif y == 'Merchant / Adelman of Manila' and z == 'Multiple business partners in Southeast Asia, China, New Spain':
        prof_edu = 'Merchant, Alderman'
        relation = 'alderman of--Manila'
        notes = 'Multiple business partners in Southeast Asia, China, New Spain'
    elif y == 'Merchant / Alderman of Manila /Ship Pilot' and z == 'Partner of Diego Hernandez Victoria':
        prof_edu = 'Merchant, Alderman, Shipman'
        relation = 'alderman of--Manila, partner of--Diego Hernández Victoria'
        notes = ''
    elif y == 'Merchant / Member of the Consulate of Commerce (1594)' and z == 'Partner with Esteban Rodriguez de Figueroa, his brother':
        prof_edu = 'Merchant, Official'
        relation = 'partner of--Esteban Rodriguez de Figueroa, brother of--Esteban Rodriguez de Figueroa'
        notes = 'Member of the Consulate of Commerce (1594)'
    elif y == 'Merchant / Mine owner' and z == 'Brother of Catalina enriquez. Brother in Law of Manuel de Lucena':
        prof_edu = 'Merchant, Miner'
        relation = 'brother of--Catalina Enriquez, brother-in-law of--Manuel de Lucena'
        notes = ''
    elif y == 'Merchant / Shipowner/' and z == 'Business Partner':
        prof_edu = 'Merchant, Shipman'
        relation = ''
        notes = ''
    elif y == 'Merchant / Shipowner/' and z == 'Partner of  Bartolomeu Vaz Landeiro':
        prof_edu = 'Merchant, Shipman'
        relation = 'partner of--Bartolomé Báez Landero'
        notes = ''
    elif y == 'Merchant / Shipowner/ Miner' and z == '':
        prof_edu = 'Merchant, Shipman, Miner'
        relation = ''
        notes = ''
    elif y == 'Merchant /Agricultor' and z == 'Arrived as colonizer with governor Luis de Carvajal (1580)':
        prof_edu = 'Merchant, Farmer'
        relation = 'colonizer with--Luis de Carvajal'
        notes = 'Arrived as colonizer with governor Luis de Carvajal (1580)'
    elif y == 'Merchant /Alderman Macao' and z == 'Legal Proxy BVL':
        prof_edu = 'Merchant, Alderman'
        relation = 'alderman of--Macao, legal proxy of--Bartolomé Báez Landero'
        notes = ''
    elif y == 'Merchant in the mining area of Pachuca' and z == 'Brother of Catalina Enriquez. Brother in Law of Manuel de Lucena':
        prof_edu = 'Merchant'
        relation = 'brother of--Catalina Enriquez, brother-in-law of--Manuel de Lucena'
        notes = 'Merchant in the mining area of Pachuca'
    elif y == 'Merchant os slaves in Cartagena' and z == 'Firend of Luis de Carvajal the younger, Antonio Díaz de Cáceres (1593, Acapulco)':
        prof_edu = 'Merchant'
        relation = 'friend of--Luis -the younger- Carvajal, friend of--Antonio Díaz de Cáceres'
        notes = 'Merchant of slaves in Cartagena, friend of Luis de Carvajal the younger and Antonio Díaz de Cáceres (1593, Acapulco)'
    elif y == 'Merchants' and z == 'Partner with Diego Hernandez Victoria; spouse Lucena, Isabel de':
        prof_edu = 'Merchant'
        relation = 'partner of--Diego Hernández Victoria, spouse of--Isabel de Lucena'
        notes = ''
    elif y == 'Merchants' and z == 'Partner with Diego Hernandez Victoria; spouse López Enríquez, Francisco':
        prof_edu = 'Merchant'
        relation = 'partner of--Diego Hernández Victoria, spouse of--Francisco López Enríquez'
        notes = ''
    elif y == 'Military commander in Manila' and z == 'Brother-in-law of Diego Hernández Victoria, Executor of his will':
        prof_edu = 'Official'
        relation = 'officer in--Manila, brother-in-law of--Diego Hernández Victoria, executor of the will of--Diego Hernández Victoria'
        notes = 'Military commander in Manila'
    elif y == 'Mine owner' and z == 'Husband of Catalina Enriquez':
        prof_edu = 'Miner'
        relation = 'spouse of--Catalina Enriquez'
        notes = ''
    elif y == 'Miner' and z == '':
        prof_edu = 'Miner'
        relation = ''
        notes = ''
    elif y == 'Official /Ship Pilot / Shipowner / Merchant' and z == 'Business Partner with BVL':
        prof_edu = 'Official, Shipman, Merchant'
        relation = 'partner of--Bartolomé Báez Landero'
        notes = ''
    elif y == 'Priest' and z == 'Named Diego Hernández Victoria as executor of his will':
        prof_edu = 'Priest'
        relation = 'Diego Hernández Victoria++executor of the will of--' + w
        notes = ''
    elif y == 'Priest / Dean of the Cathedral of Mexico /Merchant' and z == 'Business partner with Diego Fernandez Victoria':
        prof_edu = 'Priest, Dean, Merchant'
        relation = 'dean of--Cathedral of Mexico, partner of--Diego Hernández Victoria'
        notes = ''
    elif y == 'Public officer in Manila' and z == '':
        prof_edu = 'Official'
        relation = 'officer in--Manila'
        notes = ''
    elif y == 'Roman Catholic Priest / Merchant' and z == 'Business Partners with Diego Hernández Victoria':
        prof_edu = 'Priest, Merchant'
        relation = 'partner of--Diego Hernández Victoria'
        notes = 'Roman Catholic Priest'
    elif y == 'Ship Pilot' and z == 'Captaiin of Bartolomeu Vaz Landeiro, nephew':
        prof_edu = 'Shipman'
        relation = 'Bartolomé Báez Landero++uncle of--' + w
        notes = ''
    elif y == 'Ship Pilot' and z == 'Pilot at the service of Bartolomeu Vaz Landeiro':
        prof_edu = 'Shipman'
        relation = 'servant of--Bartolomé Báez Landero'
        notes = ''
    elif y == 'Wife of Diego Hernández Victoria' and z == '':
        prof_edu = 'Housewife'
        relation = 'spouse of--Diego Hernández Victoria'
        notes = ''
    elif y == 'hosewife' and z == 'Wife of  Antonio Díaz de Cáceres':
        prof_edu = 'Housewife'
        relation = 'spouse of--Antonio Díaz de Cáceres'
        notes = ''
    elif y == 'housewife, sister of Luis de Carvajal' and z == 'Mother-in-law of Jorge de Almeida and Antonio Díaz de Cáceres':
        prof_edu = 'Housewife'
        relation = 'mother-in-law of--Jorge de Almeida, mother-in-law of--Antonio Díaz de Cáceres'
        notes = ''
    else:
        prof_edu = y
        relation = z
        notes = ''
        print('ERROR.')
    return pd.Series((prof_edu, relation, notes))

df_aux = pd.concat([df, df2.FULLNAME], axis=1)
df2[['PROFESSION_EDUCATION', 'RELATION', 'PROF_EDU_REL_NOTES']] = df_aux.apply(prof_edu_rel_notes, axis=1)

In [11]:
def contacts(x):
    if x == '':
        contacts = ''
        notes = ''
    elif x == 'António Díaz de Cáceres, Luis de Carvajal, Luis de Carvajal the younger, Francisca de Carvajal, ':
        contacts = 'Antonio Díaz de Cáceres, Luis Carvajal, Luis -the younger- Carvajal, Francisca Carvajal'
        notes = ''
    elif x == 'Bartolom Báez Landero':
        contacts = 'Bartolomé Báez Landero'
        notes = ''
    elif x == 'Bartolomé Báez Landero':
        contacts = 'Bartolomé Báez Landero'
        notes = ''
    elif x == 'Diego Caballero Bazan, Diego Hernández Victoria.':
        contacts = 'Diego Caballero Bazán, Diego Hernández Victoria'
        notes = ''
    elif x == 'Diego Hernández Victoria':
        contacts = 'Diego Hernández Victoria'
        notes = ''
    elif x == "Diego Hernández Victoria (Oliveira's letter)":
        contacts = 'Diego Hernández Victoria'
        notes = "Oliveira's letter"
    elif x == "Diego Hernández Victoria (Oliveira's letter), Francisco de Novaes, Antonio Rrabello":
        contacts = 'Diego Hernández Victoria, Francisco de Novaes, Antonio Rrabello Bravo'
        notes = "Oliveira's letter"
    elif x == 'Diego Hernández Victoria, Diego Caballero Bazan, Francisco de Paz, Alonso Martín':
        contacts = 'Diego Hernández Victoria, Diego Caballero Bazán, Francisco de Paz, Alonso Martin'
        notes = ''
    elif x == 'Diego Hernández Victoria, Fernando de Castro':
        contacts = 'Diego Hernández Victoria, Fernando de Castro'
        notes = ''
    elif x == 'Diego Hernández Victoria, Francisco de Espartero':
        contacts = 'Diego Hernández Victoria, Francisco de Rodríguez de Espartero'
        notes = ''
    elif x == 'Diego Hernández Victoria, Francisco de Paz, Diego Caballero Bazán':
        contacts = 'Diego Hernández Victoria, Francisco de Paz, Diego Caballero Bazán'
        notes = ''
    elif x == 'Diego Hernández Victoria, María de Zárate, Joan de Bustamante, Diego de Valdés, Juan Juárez Gallinato':
        contacts = 'Diego Hernández Victoria, María de Zárate, Joan de Bustamante, Diego de Valdés, Juan Juárez Gallinato'
        notes = ''
    elif x == 'Diego Hernández Victoria, María de Zárate, Juan Juárez Gallinato, Joan de Bustamante, Luis de Balmaceda, Diego Hernández Victoria (nephew)':
        contacts = 'Diego Hernández Victoria, María de Zárate, Juan Juárez Gallinato, Joan de Bustamante, Luis de Balmaceda'
        notes = ''
    elif x == 'Diego Hernández Victoria, María de Zárate, Luis de Balmaceda, Diego de Valdés, Juan Juárez Gallinato':
        contacts = 'Diego Hernández Victoria, María de Zárate, Luis de Balmaceda, Diego de Valdés, Juan Juárez Gallinato'
        notes = ''
    elif x == 'Diego Hernández Victoria, captain Joan de Alzesa':
        contacts = 'Diego Hernández Victoria, Joan de Alzesa'
        notes = ''
    elif x == 'Diogo Fernandes Vitoria':
        contacts = 'Diego Hernández Victoria'
        notes = ''
    elif x == 'Fernando de Castro, Diego Hernández Victoria':
        contacts = 'Fernando de Castro, Diego Hernández Victoria'
        notes = ''
    elif x == 'Francisco de Paz, Diego Caballero Bazán,  Canon Luis de Barcelos, Thomé Nieto, Miguel Riveiro, Antonio Leitāo, Fernando Peres, Diogo Lopes, Diogo Jorge, Diogo Dias Neto, Bernardo de Luna, Jeronimo Tyrado, Juliāo Gomez, Antonio Garcês, Mestre Xrovāo Anbrozio,  Joā de Guzmāo, Diogo deasmudio (Zamudio), Joāo Vaz':
        contacts = 'Francisco de Paz, Diego Caballero Bazán, Canon Luis de Barcelos, Thomé Nieto, Miguel Riveiro, António Leitāo, Fernando Peres, Diogo Lopes, Diogo Jorge, Diogo Díaz Neto, Bernardo de Luna, Jeronimo Tyrado, Juliāo Gomez, Antonio Garcês, Mestre Xrováo Anbrozio, Joāo de Guzmāo, Diogo Zamudio, Joāo Vaz'
        notes = ''
    elif x == 'Francisco de Paz, Diego Hernández Victoria':
        contacts = 'Francisco de Paz, Diego Hernández Victoria'
        notes = ''
    elif x == 'Jorge de Almeida, ':
        contacts = 'Jorge de Almeida'
        notes = ''
    elif x == 'Jorge de de Almeida, Antonio Díaz de Cáceres':
        contacts = 'Jorge de Almeida, Antonio Díaz de Cáceres'
        notes = ''
    elif x == 'Letter of Joāo de Olveira (1584)':
        contacts = 'Joāo Oliveira'
        notes = 'Letter of Joāo de Olveira (1584)'
    elif x == 'Luis de Carmelo':
        contacts = 'Luis de Carmelo'
        notes = ''
    elif x == 'Luis de Carvajal, Luis de Carvajal the younger, Francisca de carvajal':
        contacts = 'Luis Carvajal, Luis -the younger- Carvajal, Francisca Carvajal'
        notes = ''
    elif x == 'Manuel Lucena, Jorge de Almeida, ':
        contacts = 'Manuel de Lucena, Jorge de Almeida'
        notes = ''
    elif x == 'Manuel Lucena, Jorge de Almeida,  Antonio Díaz de Cáceres':
        contacts = 'Manuel de Lucena, Jorge de Almeida, Antonio Díaz de Cáceres'
        notes = ''
    elif x == 'Martín de Saravia, merchant, partner of Diego Hernández Vitoria':
        contacts = 'Martín de Saravia, Diego Hernández Victoria'
        notes = ''
    elif x == 'Partner of Diego Hernández Victoria':
        contacts = 'Diego Hernández Victoria'
        notes = ''
    elif x == 'Pedro Gómez Reynel and Juan Rodriguez Couthino, asentistas (slavers( in the Atlantic':
        contacts = 'Pedro Gómez Reynel, Juan Rodriguez Couthino'
        notes = ''
    elif x == 'Sebastiāo Jorge, Antonio Vaz, Henrique Borges, Melchior Correia, Antonio Rrabello Bravo, Fernán Soberas, Father Nuno Fernandez, Antonio Garcês, Antonio Teixeira Lobo, Damiā Gonçalvez, Antonio Vieira, Pedro de Brito.':
        contacts = 'Sebastiāo Jorge Moxar, Antonio Vaz, Henrique Borges, Melchior Correia, Antonio Rrabello Bravo, Fernán Soberas, Father Nuno Fernandez, Antonio Garcês, Antonio Teixeira Lobo, Damiāo Gonçalves, Antonio Vieira, Pedro de Brito'
        notes = ''
    elif x == '[Letter Canton, 1584] Jeronimo Tyrado, Juliāo Gomez, Antonio Garcês, Mestre Xrovão Anbrozio, Diogo Cavaleiro Baçāo, Joāo de Guzmāo, Digo Desamudio (de Zamudio), Jõao Vaz':
        contacts = 'Jeronimo Tyrado, Juliāo Gomez, Antonio Garcês, Mestre Xrováo Anbrozio, Diego Caballero Bazán, Joāo de Guzmāo, Diogo Zamudio, Joāo Vaz'
        notes = 'Letter Canton, 1584'
    elif x == 'captain Fernando de Castro, Diego Hernández Victoria.':
        contacts = 'Fernando de Castro, Diego Hernández Victoria'
        notes = ''
    else:
        print('ERROR.')
    return pd.Series((contacts, notes))
    
df2[['CONTACTS', 'CONTACT_NOTES']] = df.CONTACTS.apply(contacts)

In [12]:
contacts_set = set()
for contacts in df2.CONTACTS.unique():
    for contact in contacts.split(', '):
        contacts_set.add(contact)

In [13]:
relations_set = set()
for relations in df2.RELATION.unique():
    for relation in relations.split(', '):
        r = relation
        if '++' in r:
            r = ''.join(r.split('++')[1:])
        r = r.split('--')[0]
        relations_set.add(r)

In [14]:
relations_set

{'',
 'alderman of',
 'brother of',
 'brother-in-law of',
 'captain of',
 'captain-major of',
 'colonizer with',
 'cousin of',
 'dean of',
 'executor of the will of',
 'friend of',
 'governor of',
 'granted power of attorney to',
 'invested in',
 'legal proxy of',
 'mother-in-law of',
 'officer in',
 'owes to',
 'paid to',
 'partner of',
 'sent merchandise to',
 'servant of',
 'spouse of',
 'travelled to',
 'travelled with',
 'uncle of'}

In [15]:
schema = {
    'alderman of':                  {'subject': 'Person', 'object': 'Place'},
    'brother of':                   {'subject': 'Person', 'object': 'Person'},
    'brother-in-law of':            {'subject': 'Person', 'object': 'Person'},
    'captain of':                   {'subject': 'Person', 'object': 'Galleon'},
    'captain-major of':             {'subject': 'Person', 'object': 'Place'},
    'colonizer with':               {'subject': 'Person', 'object': 'Person'},
    'cousin of':                    {'subject': 'Person', 'object': 'Person'},
    'dean of':                      {'subject': 'Person', 'object': 'Institution'},
    'executor of the will of':      {'subject': 'Person', 'object': 'Person'},
    'friend of':                    {'subject': 'Person', 'object': 'Person'},
    'governor of':                  {'subject': 'Person', 'object': 'Place'},
    'granted power of attorney to': {'subject': 'Person', 'object': 'Person'},
    'invested in':                  {'subject': 'Person', 'object': 'Galleon'},
    'legal proxy of':               {'subject': 'Person', 'object': 'Person'},
    'mother-in-law of':             {'subject': 'Person', 'object': 'Person'},
    'officer in':                   {'subject': 'Person', 'object': 'Place'},
    'owes to':                      {'subject': 'Person', 'object': 'Person'},
    'paid to':                      {'subject': 'Person', 'object': 'Person'},
    'partner of':                   {'subject': 'Person', 'object': 'Person'},
    'sent merchandise to':          {'subject': 'Person', 'object': 'Person'},
    'servant of':                   {'subject': 'Person', 'object': 'Person'},
    'spouse of':                    {'subject': 'Person', 'object': 'Person'},
    'travelled to':                 {'subject': 'Person', 'object': 'Place'},
    'travelled with':               {'subject': 'Person', 'object': 'Person'},
    'uncle of':                     {'subject': 'Person', 'object': 'Person'},
}

In [16]:
profs = []
for x in df2.PROFESSION_EDUCATION:
    for y in x.split(', '):
        if y:
            profs.append(y)

In [17]:
Counter(profs).most_common()

[('Merchant', 64),
 ('Shipman', 13),
 ('Official', 6),
 ('Housewife', 5),
 ('Miner', 4),
 ('Alderman', 3),
 ('Priest', 3),
 ('Landowner', 1),
 ('Governor', 1),
 ('Captain-Major', 1),
 ('Farmer', 1),
 ('Dean', 1)]

In [18]:
professions_priority = [
    'Alderman',
    'Captain-Major',
    'Dean',
    'Priest',
    'Official',
    'Landowner',
    'Farmer',
    'Miner',
    'Shipman',
    'Merchant',
]

In [19]:
def calculate_profession(professions):
    profs = professions.split(', ')
    if len(profs) == 1:
        r = profs[0]
    else:
        r = sorted(professions.split(', '), key=lambda x: professions_priority.index(x))[0]
    return r

def calculate_type2(n):
    return g.node[n]['type2'] if n in g else 'Unknown profession'

In [20]:
g = nx.DiGraph()
for i, row in df2.iterrows():
    g.add_node(row.FULLNAME, type1='Person', type2=calculate_profession(row.PROFESSION_EDUCATION))
    if row.PLACE_ORIGIN:
        g.add_node(row.PLACE_ORIGIN, type1='Place', type2='Place')
        g.add_edge(row.FULLNAME, row.PLACE_ORIGIN, label='from')
        if row.COUNTRY_ORIGIN:
            g.add_node(row.COUNTRY_ORIGIN, type1='Place', type2='Place')
            g.add_edge(row.PLACE_ORIGIN, row.COUNTRY_ORIGIN, label='in')
    for relation in row.RELATION.split(', '):
        if relation:
            subjct = row.FULLNAME
            if '++' in relation:
                subjct, relation = relation.split('++')
            r, objct = relation.split('--')
            g.add_node(subjct, type1='Person', type2=calculate_type2(subjct))
            type2 = calculate_type2(objct) if schema[r]['object'] == 'Person' else schema[r]['object']
            g.add_node(objct, type1=schema[r]['object'], type2=type2)
            g.add_edge(subjct, objct, label=r)
    for contact in row.CONTACTS.split(', '):
        if contact and not g.has_edge(row.FULLNAME, contact):
            g.add_node(contact, type1='Person', type2=calculate_type2(contact))
            g.add_edge(row.FULLNAME, contact, label='contact of')
    for location in row.LOCATION.split(', '):
        if location:
            g.add_node(location, type1='Place', type2='Place')
            g.add_edge(row.FULLNAME, location, label='located in')

In [21]:
nx.write_gexf(g, 'data/merchant.gexf')